# Send Jobs from your Computer

<img src="imgs/Upload_Run_Download.png" alt="Procedure" height="250" title="Procedure"/>

In [ ]:
from VipSession import VipSession

## Initiate your Session 

*API key is provided in your VIP account settings*

In [ ]:
# API key can be stored in an environment variable
VipSession.init(api_key="VIP_API_KEY")

In [ ]:
# Instantiate a Session with a name
my_session = VipSession("demo-single-job")

## Upload your dataset on VIP

Upload a single folder containing the full dataset

In [ ]:
input_dir = "examples/data/quest_1job"
! tree {input_dir}

In [ ]:
my_session.upload_inputs(input_dir);

*The session is automatically backed up at the end of each step* 

In [ ]:
from pathlib import Path
! tree {Path(my_session.output_dir).parent} 

In [ ]:
! head {Path(my_session.output_dir) / "session_data.json"} 

## Launch the pipeline

### Pipeline Identifier
App/Version

In [ ]:
pipeline_id = "CQUEST/0.1.1"

### Input Settings

*Requiered Inputs (see https://vip.creatis.insa-lyon.fr/)*:

In [ ]:
input_settings = {
    "parameter_file" : "examples/data/quest_1job/parameters.txt",
    "data_file" : "examples/data/quest_1job/signal.mrui",
    "zipped_folder" : "examples/data/quest_1job/basis.zip",
}

Application launch

In [ ]:
my_session.launch_pipeline(pipeline_id, input_settings);

*See progression on https://vip.creatis.insa-lyon.fr/* ...

... or monitor the workflow from this terminal:

In [ ]:
my_session.monitor_workflows(waiting_time=2);

## Download your Results

In [ ]:
my_session.download_outputs();

*If `output_dir` is not specified, results are stored at default location*

In [ ]:
! tree {my_session.output_dir}

# Parallelize your Jobs for Bigger Datasets

*Suppose we have more signals to analyse*

In [ ]:
new_dataset = "examples/data/quest_2jobs"
! tree {new_dataset}

**_Lists_** of parameters launch **_parallel_** jobs on VIP:

In [ ]:
# Input parameters of the pipeline
new_settings = {
    "data_file" : [
        "examples/data/quest_2jobs/signals/001.mrui",
        "examples/data/quest_2jobs/signals/002.mrui",
    ],
    "parameter_file" : "examples/data/quest_2jobs/parameters.txt",
    "zipped_folder" : "examples/data/quest_2jobs/basis.zip",
}

Run a new session with the new parameters

In [ ]:
new_session = VipSession(
    session_name = "demo-parallel-jobs",
    input_dir = new_dataset,
    pipeline_id = "CQUEST/0.1.1",
    input_settings = new_settings
).run_session();

Check the output files

In [ ]:
! tree {new_session.output_dir}

# Repeat a Previous Experiment

Resume previous session "demo_single_job" and relaunch it *as is*

In [ ]:
VipSession("demo-single-job").run_session(update_files=False, waiting_time=2);

Check the output files

In [ ]:
! tree {VipSession("demo-single-job", verbose=False).output_dir}

# Remove Temporary Data from VIP

*After the download, **your input and output data are still on VIP** (https://vip.creatis.insa-lyon.fr/)*

In [ ]:
my_session.vip_input_dir, my_session.vip_output_dir

*Please remove your temporary data when a session is over.*

In [ ]:
VipSession("demo-single-job").finish()
VipSession("demo-parallel-jobs").finish()

*The output data downloaded on your computer is yours to remove*

In [ ]:
! rm -r vip_outputs

# Launch Executions from Girder Datasets

In [ ]:
from VipCI import VipCI

Input Settings

In [ ]:
my_settings = {
    "zipped_folder": "/collection/ReproVIPSpectro/data/quest/basis.zip",
    "data_file": "/collection/ReproVIPSpectro/data/quest/signals",
    "parameter_file": "/collection/ReproVIPSpectro/data/quest/parameters/quest_param_117T_B.txt",
}

Initiate connection

In [ ]:
VipCI.init(
    vip_key="VIP_API_KEY", 
    girder_key="GIRDER_API_KEY",
)

Launch & monitor execution

In [ ]:
session = VipCI(
    pipeline_id="CQUEST/0.1-logs", 
    input_settings=my_settings, 
    output_dir="/collection/ReproVIPSpectro/results/new-folder"
).run_session()